In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

/home/staff/l/lipai.huang/.conda/envs/py38/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [7]:
# find harris county
# 'date', 'lat', 'lng', 'act_idx'
db1 = pd.read_csv('0201_0228_total_daily.csv')
db1 = db1.loc[:, ~db1.columns.str.contains('^Unnamed')]
db1.head()

,geography,xlon,xlat,bounds,agg_day_period,agg_time_period,activity_index_total,geometry
0,23131022212033211,-95.411453,29.730396,"-95.41213989257812,29.72979997260222,-95.41076...",2021-02-01,1,0.027110,POINT (-95.41145324707033 29.730396233962253)
1,23131022301021112,-95.165634,29.819796,"-95.16632080078125,29.81919981751383,-95.16494...",2021-02-01,1,0.045658,POINT (-95.16563415527344 29.819795546829383)
2,23131022303302210,-95.138168,29.711314,"-95.13885498046875,29.71071768156532,-95.13748...",2021-02-01,1,0.031654,POINT (-95.13816833496094 29.711314056301678)
3,23131022012201112,-95.429306,30.029272,"-95.42999267578125,30.0286775329042,-95.428619...",2021-02-01,1,0.040890,POINT (-95.42930603027344 30.02927200987857)
4,23131022212310011,-95.378494,29.725626,"-95.37918090820312,29.72502973997891,-95.37780...",2021-02-01,1,0.077237,POINT (-95.37849426269533 29.72562602968721)


In [8]:
db2 = pd.read_excel('filtered-Entergy Sorted Zipcodes Louisiana.xlsx')
db2.head()

,Zip Code,"Aug 29, 09:29 PM","Aug 30, 09:01 PM","Aug 31, 09:15 PM","Sep 01, 09:01 PM","Sep 02, 09:15 PM","Sep 03, 09:02 PM","Sep 04, 08:38 PM","Sep 05, 09:14 PM","Sep 06, 09:16 PM",...,"Sep 14, 09:17 PM","Sep 15, 08:52 PM","Sep 16, 08:14 PM","Sep 17, 08:19 PM","Sep 18, 11:51 PM","Sep 19, 09:48 PM","Sep 20, 10:07 PM","Sep 21, 11:06 PM","Sep 22, 09:18 PM","Sep 23, 12:14 PM"
0,70001,20478,20448,20448,20471,20301,16492,14377,14104,12339,...,5,4,1,4,3,0,3,5,2,2
1,70002,11032,11029,10935,10935,10935,11116,8499,7441,4575,...,14,26,3,1,15,8,16,0,0,41
2,70003,17253,17178,17179,17108,17108,17426,17406,17127,15766,...,16,27,20,19,12,5,13,127,8,10
3,70005,13375,13336,13336,13334,13334,10928,9333,4606,5741,...,4,3,3,3,2,1,3,2,4,9
4,70006,7723,7718,7718,7599,7599,7362,7318,7055,5463,...,5,5,3,1,3,0,0,0,1,1


In [4]:
db3_dir = 'CuebiqMicro_Harvey/Harvey_Micro/48201/average'
db3_sub_fs = os.listdir(db3_dir)
db3 = pd.DataFrame()
for f in db3_sub_fs:
    if not f.endswith('.csv'):
        continue
    db3_sub_f = pd.read_csv(os.path.join(db3_dir, f))
    db3_sub_f = db3_sub_f.loc[:, ~db3_sub_f.columns.str.contains('^Unnamed')]
    if db3.empty:
        db3 = db3_sub_f
    else:
        db3 = pd.concat([db3, db3_sub_f], axis=0)
    

In [13]:
db3 = db3.groupby(['FIPS'], as_index=False).agg(dict(zip(db3.columns[1:], ['mean']*len(db3.columns[1:]))))
db3.to_csv('CuebiqMicro_Harvey.csv', index=False)
db3.head()

,FIPS,CBG_Avg_Time(min),CBG_Avg_Distance(m),CBG_Avg_Trips,CBG_Avg_ROG(m)
0,48201100000,107.296802,4018.707364,3.156020,10113.662408
1,48201210100,104.213268,6103.488950,2.784135,12248.405482
2,48201210400,87.282331,3480.476643,3.137605,7580.504221
3,48201210500,88.823493,4141.978605,2.857448,8516.787420
4,48201210600,103.030483,5121.375618,2.453517,9403.280683


In [2]:
db4_sub1 = pd.read_csv('hurricane_ida/unzip/Ida_homecbg_poi/2021-08-01.csv')
db4_sub1.head()

,home_block_group_id,date,time,place_id,count
0,10010201002,2021-08-01,4,2738065,1
1,10010206001,2021-08-01,1,3075317,1
2,10010206001,2021-08-01,2,2580868,1
3,10010206001,2021-08-01,7,3075317,1
4,10010206001,2021-08-01,11,2785536,1


In [17]:
pd.read_csv('hurricane_ida/unzip/Ida_POI_POI/2021-08-02.csv')

,cuebiq_id,date,time,begin_POI,end_POI
0,3125028711,2021-08-02,15,9025364,9025364
1,3125028711,2021-08-02,16,9025364,9025364
2,3125028711,2021-08-02,18,9025364,2796608
3,3125028711,2021-08-02,18,2796608,9023234
4,3126442631,2021-08-02,14,2824266,27373820
...,...,...,...,...,...
202313,4529501262,2021-08-02,20,9034903,9034903
202314,4529509632,2021-08-02,19,9022189,9022189
202315,4529532248,2021-08-02,22,2599447,2599447
202316,4529535375,2021-08-02,20,3035211,3035211


In [18]:
def data_agg(folder_path):
    fps = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    db = pd.DataFrame()
    for fp in fps:
        sub_db = pd.read_csv(fp)
        sub_db = sub_db.loc[:, ~sub_db.columns.str.contains('^Unnamed')]
        del sub_db['time']
        if db.empty:
            db = sub_db.copy()
        else:
            db = pd.concat([db, sub_db], axis=0)
    if 'count' in list(db.columns):
        db = db.groupby([col for col in db.columns if not col == 'count'], as_index=False).agg({'count': 'sum'})
    else:
        db = db.groupby([col for col in db.columns if not col == 'cuebiq_id'], as_index=False).agg({'cuebiq_id': 'count'}).rename(columns={'cuebiq_id': 'count'})
    return db
    

In [4]:
db_cbg_cbg = data_agg('hurricane_ida/unzip/Ida_CBG_CBG')

In [6]:
db_cbg_cbg.to_csv('Ida_CBG_CBG.csv', index=False)

In [7]:
db_home_poi = data_agg('hurricane_ida/unzip/Ida_homecbg_poi')

In [9]:
db_home_poi.to_csv('Ida_homecbg_poi.csv', index=False)

In [19]:
db_poi_poi = data_agg('hurricane_ida/unzip/Ida_POI_POI')

In [21]:
db_poi_poi.to_csv('Ida_POI_POI.csv', index=False)

In [15]:
pd.read_csv('hurricane_ida/unzip/Ida_CBG_CBG/2021-08-01.csv')

,date,time,begin_CBG,end_CBG,count
0,2021-08-01,0,220019601002,220550010012,2
1,2021-08-01,0,220019602001,220019602001,2
2,2021-08-01,0,220019602001,220019608001,2
3,2021-08-01,0,220019602002,220979611001,1
4,2021-08-01,0,220019602003,220019603002,1
...,...,...,...,...,...
319019,2021-08-01,23,221259518003,221259517011,1
319020,2021-08-01,23,221259518003,221259518003,2
319021,2021-08-01,23,221259518003,221259518004,2
319022,2021-08-01,23,221259518004,220779522002,1


# DB vis

In [37]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from tqdm import tqdm
from sklearn.metrics import silhouette_score

In [38]:
louis_df = pd.read_excel('filtered-Entergy Sorted Zipcodes Louisiana.xlsx')

In [39]:
louis_df_melt = louis_df.melt(id_vars=['Zip Code'], var_name='date', value_name='energy').rename(columns={'Zip Code': 'zip_code'})
louis_df_melt['date'] = louis_df_melt['date'].apply(lambda x: datetime.strptime(x, '%b %d, %I:%M %p'))
louis_df_melt = louis_df_melt.sort_values(by=['zip_code', 'date']).reset_index(drop=True)

In [21]:
louis_df_melt.dtypes

zip_code             int64
date        datetime64[ns]
energy               int64
dtype: object

In [40]:
X = louis_df_melt['energy'].to_numpy().reshape(-1,1)
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
labels = kmeans.predict(X)
louis_df_melt['label_4'] = labels

In [31]:
louis_df_melt

,zip_code,date,energy,label_4
0,70001,1900-08-29 21:29:00,20478,2
1,70001,1900-08-30 21:01:00,20448,2
2,70001,1900-08-31 21:15:00,20448,2
3,70001,1900-09-01 21:01:00,20471,2
4,70001,1900-09-02 21:15:00,20301,2
...,...,...,...,...
8133,71483,1900-09-19 21:48:00,0,0
8134,71483,1900-09-20 22:07:00,0,0
8135,71483,1900-09-21 23:06:00,0,0
8136,71483,1900-09-22 21:18:00,0,0


In [41]:
silhouette_score(X, labels)

0.8773929089614471

In [24]:
louis_df_melt['date'] = louis_df_melt['date'].apply(lambda x: x.timestamp())
X = louis_df_melt[['date', 'energy']].values

In [25]:
thresholds = [0.5, 1, 1.5]

In [28]:
clusters = []
for i in tqdm(range(len(X)), desc='clustering'):
    cluster = [X[i]]
    for j in range(len(clusters)):
        distance, _ = fastdtw(X[i], clusters[j][0], dist=euclidean)
        if distance < thresholds[0]:
            clusters[j].append(X[i])
            break
    else:
        clusters.append(cluster)
score = silhouette_score(X, clusters)

clustering: 100%|██████████| 8138/8138 [07:17<00:00, 18.59it/s]
/home/staff/l/lipai.huang/.conda/envs/py38/lib/python3.8/site-packages/sklearn/utils/validation.py:1137: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = np.asarray(y)


ValueError: Found input variables with inconsistent numbers of samples: [8138, 2664]

In [ ]:
louis_df_melt['label_dtw_0.5'] = clusters
louis_df_melt

In [ ]:
score

In [2]:
micro_df = pd.read_csv('CuebiqMicro_Harvey.csv')
micro_df.head()

,FIPS,CBG_Avg_Time(min),CBG_Avg_Distance(m),CBG_Avg_Trips,CBG_Avg_ROG(m)
0,48201100000,107.296802,4018.707364,3.156020,10113.662408
1,48201210100,104.213268,6103.488950,2.784135,12248.405482
2,48201210400,87.282331,3480.476643,3.137605,7580.504221
3,48201210500,88.823493,4141.978605,2.857448,8516.787420
4,48201210600,103.030483,5121.375618,2.453517,9403.280683


In [5]:
micro_df.dtypes

FIPS                     int64
CBG_Avg_Time(min)      float64
CBG_Avg_Distance(m)    float64
CBG_Avg_Trips          float64
CBG_Avg_ROG(m)         float64
dtype: object

In [7]:
X = micro_df.drop('FIPS', axis=1)
X = X.apply(lambda x: x - x.mean())
X = X.apply(lambda x: x / x.std())

In [9]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
labels = kmeans.predict(X)

In [13]:
micro_df['label_4'] = labels
micro_df.head()

,FIPS,CBG_Avg_Time(min),CBG_Avg_Distance(m),CBG_Avg_Trips,CBG_Avg_ROG(m),label_4
0,48201100000,107.296802,4018.707364,3.156020,10113.662408,0
1,48201210100,104.213268,6103.488950,2.784135,12248.405482,2
2,48201210400,87.282331,3480.476643,3.137605,7580.504221,3
3,48201210500,88.823493,4141.978605,2.857448,8516.787420,0
4,48201210600,103.030483,5121.375618,2.453517,9403.280683,0


In [36]:
ida_cbg_cbg_df = pd.read_csv('Ida_CBG_CBG.csv')
ida_cbg_cbg_df

,date,begin_CBG,end_CBG,count
0,2021-08-01,220019601001,220019601001,30
1,2021-08-01,220019601001,220019601002,4
2,2021-08-01,220019601001,220019601003,5
3,2021-08-01,220019601001,220019601004,20
4,2021-08-01,220019601001,220019601005,14
...,...,...,...,...
9700305,2021-09-30,221259518004,221259517021,2
9700306,2021-09-30,221259518004,221259518001,10
9700307,2021-09-30,221259518004,221259518002,28
9700308,2021-09-30,221259518004,221259518003,62


In [4]:
ida_cbg_cbg_df.dtypes

date         object
begin_CBG     int64
end_CBG       int64
count         int64
dtype: object

In [32]:
pd.read_csv('Ida_homecbg_poi.csv')

,home_block_group_id,date,place_id,count
0,10010201002,2021-08-01,2738065,1
1,10010201002,2021-08-06,9085723,1
2,10010201002,2021-08-06,29167162,1
3,10010202001,2021-09-01,2649100,1
4,10010202001,2021-09-03,9094173,1
...,...,...,...,...
7660162,560459511002,2021-09-25,2684454,1
7660163,560459511002,2021-09-27,9032918,1
7660164,560459511002,2021-09-29,27300245,1
7660165,560459511002,2021-09-29,29254640,1


In [33]:
# agg begin_POI -> sum: count, rename: out_degree
# date as unit
# 
pd.read_csv('Ida_POI_POI.csv')

,date,begin_POI,end_POI,count
0,2021-08-01,2459140,2535374,22
1,2021-08-01,2459185,2462992,1
2,2021-08-01,2459185,2916770,1
3,2021-08-01,2459185,3111442,1
4,2021-08-01,2459185,3286294,1
...,...,...,...,...
3795736,2021-09-30,30178383,29294692,1
3795737,2021-09-30,30178444,30178444,1
3795738,2021-09-30,30178456,30178456,1
3795739,2021-09-30,30178474,9023131,1
